## Imports

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers 
import os
from PIL import Image

#from tensorflow_docs.vis import embed
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import imageio


import seaborn as sns
sns.set_theme(font_scale=1.6, style='whitegrid') 

## Constants and hyperparameters

In [ ]:
batch_size = 32
num_channels = 3
num_classes = 3
image_size = 64
latent_dim = 128

In [ ]:
types = ['Grass', 'Fire', 'Water']
allowed_img_types = ['jpeg', 'png', 'jpg']

IMG_SIZE = 64
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
BATCH_SIZE = 32
Z_DIM = 100
NUM_CLASSES = 3

poke_img_path  = '../input/3class-pokemon/Poke_img'
#poke_stat_path = '../input/pokemon-images-and-types/pokemon.csv'
#satellite_path = '../input/landscape-classification/intel-image-classification/train'

#pokemon_data = pd.read_csv(poke_stat_path)
#pokemon_data.head(2)

## Loading the dataset and preprocessing it

In [ ]:
class_names = ['Fire', 'Water', 'Grass']

In [ ]:
train_images = []
train_labels = []
for style in os.listdir(poke_img_path): 
        for poke in os.listdir(poke_img_path+'/'+str(style)):
            train_labels.append(class_names.index(str(style)))
            img = Image.open(poke_img_path+'/'+str(style)+'/' + str(poke)).convert('RGBA')
            background = Image.new("RGBA", img.size, (255, 255, 255))
            img = Image.alpha_composite(background, img).convert('RGB')
            img = img.resize((64,64))
            train_images.append(np.asarray(img))

train_images = np.reshape(train_images,(-1,64,64,3))
train_images = train_images.astype(np.float32)
train_images = train_images / 255.
integ_labels = np.asarray(train_labels)
train_labels = np.asarray(train_labels)

train_labels = keras.utils.to_categorical(train_labels, 3)

train_images.shape, train_labels.shape


print(f"Shape of training images: {train_images.shape}")
print(f"Shape of training labels: {train_labels.shape}")

train_labels[2]

In [ ]:
batch_size = 32
BUFFER_SIZE = 1064

# a bit of data augmentation
data_augmentation = tf.keras.models.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)


def prepare(ds, shuffle=True, augment=True):
  #shuffle elements
  if shuffle:
    ds = ds.shuffle(6000)
  # Batch all datasets.
  ds = ds.batch(batch_size)
  # data augmentation 
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=BUFFER_SIZE)
  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=BUFFER_SIZE)


dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = prepare(dataset)

In [ ]:
def show_random_sample(dataset):
    plt.figure(figsize=(12, 12))
    for images, labels in dataset.take(1):
        np_imgs = images.numpy()
        np_labs = labels.numpy()

    for i in range(20):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(np_imgs[i])
        plt.title(class_names[np.argmax(np_labs[i])])
        plt.axis("off")
    plt.tight_layout()
    plt.savefig('exsample.pdf')
    plt.show()

show_random_sample(dataset)

## Calculating the number of input channel for the generator and discriminator

In [ ]:
generator_in_channels = latent_dim + num_classes        # input of G
discriminator_in_channels = num_channels + num_classes  # input of D
print(generator_in_channels, discriminator_in_channels)

## Creating the discriminator and generator

In [ ]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((64, 64, discriminator_in_channels)),
        layers.Conv2D(512, (3, 3), strides=(1, 1), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, (3, 3), strides=(1, 1), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(1, 1), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, (3, 3), strides=(1, 1), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 7x7x(128 + num_classes) map.
        layers.Dense(8 * 8 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((8, 8, generator_in_channels)),
        layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, (4, 4), padding="same", activation="sigmoid"),
    ],
    name="generator",
)


generator.summary()
discriminator.summary()

### A print function

In [ ]:
NUM_CLASSES = 3
r, c = 5, NUM_CLASSES

z_input = tf.random.normal(shape=(r*c, latent_dim))
labels = np.asarray([0,0,0,0,0,1,1,1,1,1,2,2,2,2,2])
labels = keras.utils.to_categorical(labels, 3)
tot_noise = tf.concat([z_input, labels], axis=1)

def save_image(generator, iteration, tot_noise):
    NUM_CLASSES = 3
    r, c = 5, NUM_CLASSES
    
    test_imgs = generator(tot_noise, training=True)
    gen_image = (test_imgs * 0.5) + 0.5
    count = 0
    plt.figure(figsize = (15,7))
    for i in range(5 * NUM_CLASSES):
        plt.subplot(NUM_CLASSES, 5, i+1)
        img_test = tf.keras.preprocessing.image.array_to_img(gen_image[i])
        plt.imshow(img_test)
        plt.axis('off')
        if (i == 0 or i == 5 or i == 10): 
            plt.ylabel(class_names[count])
            count +=1
    plt.tight_layout()
    plt.savefig('./iteration'+ str(iteration)+'.png')
    plt.show()
        
    plt.close()


def show_losses(d_loss, g_loss, iteration):
    plt.figure(figsize=(10,4))
    plt.plot(g_loss, lw=2, alpha=0.6, color='red', label='G')
    plt.plot(d_loss, lw=2, alpha=0.6, color='blue', label='D')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Loss value')
    plt.title('Losses')
    plt.savefig('./Generated/losses'+str(iteration)+'.png')
    plt.show()
    plt.close()

## Creating a `ConditionalGAN` model

In [ ]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")
    

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer           #Adam
        self.g_optimizer = g_optimizer           #Adam
        self.loss_fn = loss_fn                   #BinaryCrossentropy

    def train_step(self, data):
  
        # Unpack the data.
        real_images, one_hot_labels = data  #images e labels (specified in the fit method)
  
        # real images-labels
        image_one_hot_labels = one_hot_labels[:, :, None, None]

        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )  #obtained a (64,64,3) tensor
 
        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]  #immagini nel batch
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim)) #batch random points of dimension Z
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        ) #all the input infos

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat([fake_image_and_labels, real_image_and_labels], axis=0)

        # Assemble labels discriminating real from fake images.
        # elenco del vero o falso
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        #self.save_images()

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)


        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }


## Training the Conditional GAN

In [ ]:
cond_gan = ConditionalGAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)

### Scheduled $\eta$ exponential decay

In [ ]:
initial_learning_rate = 0.0005

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=50,
    decay_rate=0.99,
    staircase=True)


cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5),
    g_optimizer=keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

### Training

To do: define a callbacks list

In [ ]:
save_image(cond_gan.generator, 9999, tot_noise)
steps = 500
g_loss = []
d_loss = []

for i in range(steps):
    cond_gan.fit(dataset, epochs=1)
    d_loss.append(cond_gan.disc_loss_tracker.result())
    g_loss.append(cond_gan.gen_loss_tracker.result())
    print(d_loss[i])
    save_image(cond_gan.generator, i+1, tot_noise)
    show_losses(d_loss, g_loss, (i+1))

### Save files (for kaggle)

In [ ]:
import shutil
shutil.make_archive('Generated', 'zip', './Generated')

In [ ]:
for i in range(10):
    cond_gan.fit(dataset, epochs=1) 
    save_image(cond_gan.generator, iteration=99999)

## Interpolating between classes with the trained generator (from keras tutorial)

In [ ]:
# We first extract the trained generator from our Conditiona GAN.
trained_gen = cond_gan.generator 

# Choose the number of intermediate images that would be generated in
# between the interpolation + 2 (start and last images).
num_interpolation =   10 # @param {type:"integer"} 
 
# Sample noise for the interpolation.
interpolation_noise = tf.random.normal(shape=(1, latent_dim))
interpolation_noise = tf.repeat(interpolation_noise, repeats=num_interpolation)
interpolation_noise = tf.reshape(interpolation_noise, (num_interpolation, latent_dim))


def interpolate_class(first_number, second_number):
    # Convert the start and end labels to one-hot encoded vectors.
    first_label = keras.utils.to_categorical([first_number], num_classes)
    second_label = keras.utils.to_categorical([second_number], num_classes)
    first_label = tf.cast(first_label, tf.float32)
    second_label = tf.cast(second_label, tf.float32)

    # Calculate the interpolation vector between the two labels.
    percent_second_label = tf.linspace(0, 1, num_interpolation)[:, None]
    percent_second_label = tf.cast(percent_second_label, tf.float32)
    interpolation_labels = (
        first_label * (1 - percent_second_label) + second_label * percent_second_label
    )

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([interpolation_noise, interpolation_labels], 1)
    fake = trained_gen.predict(noise_and_labels)
    return fake


start_class = 0  # @param {type:"slider", min:0, max:9, step:1}
end_class = 1  # @param {type:"slider", min:0, max:9, step:1}

fake_images = interpolate_class(start_class, end_class)

Here, we first sample noise from a normal distribution and then we repeat that for
`num_interpolation` times and reshape the result accordingly.
We then distribute it uniformly for `num_interpolation`
with the label indentities being present in some proportion.

### Saving the gif

In [ ]:
fake_images *= 255.0
converted_images = fake_images.astype(np.uint8)
converted_images = tf.image.resize(converted_images, (96, 96)).numpy().astype(np.uint8)
imageio.mimsave("animation_fire_to_water.gif", converted_images, fps=1)

### Saving weights

In [ ]:
cond_gan.generator.save_weights('generator.h5')
cond_gan.discriminator.save_weights('discriminator.h5')